# Ejercicio 3 - Trasponer matriz con coalescencia usando memoria compartida

Partiendo del código suministrado:

* Completar los fragmentos indicados con `TODO` 

* Comparar las prestaciones de ambos kernel

In [10]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [11]:
from numba import cuda, types as numba_types
import numpy as np

n = 4096*4096 # 16M elementos a procesar

# KERNEL: SIN MEMORIA COMPARTIDA + acceso NO COALESCENTE
@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)
    transposed[x][y] = a[y][x]

# KERNEL: MEMORIA COMPARTIDA + acceso COALESCENTE
@cuda.jit
def tile_transpose(a, transposed):
    # bloques 32x32

    # ARRAY en memoria compartida (accesible por todos los hilos de un mismo bloque)
    tile = cuda.shared.array(shape=(32, 32), dtype = numba_types.float32) # reserva espacio

    # ÍNDICES globales
    a_row, a_col = cuda.grid(2)

    # CARGAR de mem GLOBAL a mem COMPARTIDA, con índices locales
    tile[cuda.threadIdx.x, cuda.threadIdx.y] = a[a_row, a_col]  # warps se acceden por columna (columna lee fila de a)        
    
    # ESPERA a que todos los hilos del bloque actualicen la escritura
    cuda.syncthreads()

    # ESCRIBIR de mem COMPARTIDA a mem GLOBAL, transponiendo
    transposed[a_col, a_row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]


# PARÁMETROS
threads_per_block = (32, 32) # 2D blocks
blocks = (128, 128) #2D grid

# VARIABLES CPU
h_a = np.arange(n).reshape((4096,4096)).astype(np.float32)

# VARIABLES GPU
d_a = cuda.to_device(h_a)
d_transposed = cuda.device_array(shape=(4096,4096), dtype=np.float32) # guarda espacio

# LANZA KERNEL transpose
transpose[blocks, threads_per_block](d_a, d_transposed)
result_transpose = d_transposed.copy_to_host() # result a CPU

# LANZA KERNEL tile_transpose (optimización)
tile_transpose[blocks, threads_per_block](d_a, d_transposed)
result_tile_transpose = d_transposed.copy_to_host() # result a CPU

# RESULTADO ESPERADO
expected = h_a.T

# COMPROBACIÓN
np.testing.assert_equal(result_transpose, expected)
np.testing.assert_equal(result_tile_transpose, expected)

# MEDICIÓN
# cpu
print("\nTiempo en CPU:")
%timeit h_a.T

# gpu no optimizado
print("\nTiempo en GPU - NO optimizado:")
%timeit transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

# gpu optimizado
print("\nTiempo en GPU - OPTIMIZADO:")
%timeit tile_transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()


Tiempo en CPU:
111 ns ± 0.951 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Tiempo en GPU - NO optimizado:
1.74 ms ± 6.25 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Tiempo en GPU - OPTIMIZADO:
1.82 ms ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
